# NaijaLingo TTS SDK Tests

This notebook exercises all TTS pipelines via the Speech Service:
- Finetuned (NaijaLingo finetuned models)
- Open-source MMS TTS (Facebook)
- Gemini TTS (requires GEMINI_API_KEY configured on the service)



In [ ]:
import sys
sys.path.insert(0, "/home/chukypedro15/Naijalingo/NaijaLingoKit/src")

from naijalingo_tts import SpeechServiceTTSClient, SpeechServiceConfig, suppress_all_logs

# Configure base URL if needed (defaults to http://localhost:8000)
BASE_URL = "http://localhost:8000"

suppress_all_logs()
client = SpeechServiceTTSClient(SpeechServiceConfig(base_url=BASE_URL))
print("Service URL:", client.config.base_url)


In [ ]:
# Helper to download generated files from the speech service
import requests
from pathlib import Path

def download_tts_file(file_path: str, base_url: str = BASE_URL, local_dir: str = "downloads") -> Path:
    filename = Path(file_path).name
    url = f"{base_url.rstrip('/')}/api/tts/download/{filename}"
    out_dir = Path(local_dir)
    out_dir.mkdir(parents=True, exist_ok=True)
    local_path = out_dir / filename
    r = requests.get(url, timeout=180)
    r.raise_for_status()
    with open(local_path, "wb") as f:
        f.write(r.content)
    print("Downloaded:", local_path)
    return local_path


## Finetuned TTS: voices and models


In [ ]:
voices = client.get_finetuned_voices()
models = client.get_finetuned_models()
voices, models


## Finetuned TTS: synthesize one file


In [ ]:
resp_ft = client.synthesize_finetuned(
    text="Ndeewo. Kedu ka i mere? Ahụ dị m mma. Afọ juru m n'ihi na anyị ga-eme njem gaa Abakaliki echi. Anyị ga-ahụ ebe ndị mara mma.",
    model_name="chukypedro/igbo-tts-400m-0.3-v3-pt",
    speaker_id="Uchechi",
    device="cpu",
    output_filename="Uchechi.wav",
)


In [ ]:
# Download the finetuned output from the service and play locally
from IPython.display import Audio as APlay
import soundfile as sf

local_ft = download_tts_file(resp_ft["file_path"], base_url=BASE_URL)
audio, sr = sf.read(str(local_ft))
APlay(audio, rate=sr)

In [ ]:
resp_ft = client.synthesize_finetuned(
    text="Ndeewo. Kedu ka i mere? Ahụ dị m mma. Afọ juru m n'ihi na anyị ga-eme njem gaa Abakaliki echi. Anyị ga-ahụ ebe ndị mara mma.",
    speaker_id="Chukwuka",
    device="cpu",
    output_filename="Chukwuka.wav",
)



In [ ]:
# Download the finetuned output from the service and play locally
from IPython.display import Audio as APlay
import soundfile as sf

local_ft = download_tts_file(resp_ft["file_path"], base_url=BASE_URL)
audio, sr = sf.read(str(local_ft))
APlay(audio, rate=sr)

In [ ]:
resp_ft = client.synthesize_finetuned(
    text="Ndeewo. Kedu ka i mere? Ahụ dị m mma. Afọ juru m n'ihi na anyị ga-eme njem gaa Abakaliki echi. Anyị ga-ahụ ebe ndị mara mma.",
    model_name="chukypedro/igbo-tts-400m-0.3-v3-pt",
    speaker_id="Adaeze",
    device="cpu",
    output_filename="Adaeze.wav",
)


In [ ]:
# Download the finetuned output from the service and play locally
from IPython.display import Audio as APlay
import soundfile as sf

local_ft = download_tts_file(resp_ft["file_path"], base_url=BASE_URL)
audio, sr = sf.read(str(local_ft))
APlay(audio, rate=sr)

### Streaming TTS (SSE)
This example shows how to consume server-sent events from the NaijaLingo TTS streaming endpoint.
Events: `start`, `chunk`, `done`, `error`. Replace the text/model/speaker as needed.

In [ ]:
stream = client.synthesize_finetuned_stream(
    text=(
        "Najeriya ƙasa ce mai yalwa da al’adu, harsuna, da ƙabilu masu yawa, "
        "tana ɗauke da tarihin da ya haɗa mutane daban-daban cikin manufa guda. "
        "Daga arewa mai faɗaɗɗen hamada zuwa kudu mai arzikin gandun daji, ƙasar "
        "na cike da ni’imomi da baiwa ta halitta. Mutanen Najeriya suna da jajircewa, "
        "basira, da ƙwazo wajen fuskantar sauye-sauyen rayuwa. Duk da kalubale, ana "
        "ganin ƙarfin matasa, cigaban fasaha, da ƙoƙarin haɗin kai suna haskaka makoma "
        "mai kyau. Najeriya na ci gaba da ƙarfafa murya a nahiyar Afirka, tana nuni "
        "da dama da fata ga al’umma masu buri da mafarkai."
    ),
    model_name="chukypedro/hausa-tts-400m-0.3-pt",
    speaker_id="bashir",  # case-insensitive
    device="cpu",
    output_filename="tts_naijalingo_sample.wav",
    yield_audio_bytes=False,  # set True to get chunk audio bytes
)

for evt in stream:
    et = evt.get("event")
    data = evt.get("data", {})
    if et == "start":
        print(f"[start] chunks={data.get('chunks')} file={data.get('filename')}")
    elif et == "chunk":
        idx = data.get("index")
        total = data.get("total")
        text_snippet = (data.get("text") or "")[:80].replace("\n", " ")
        print(f"[chunk] {idx}/{total} text='{text_snippet}…'")
    elif et == "done":
        print(f"[done] saved={data.get('filename')} elapsed={data.get('elapsed_sec')}s")
    elif et == "error":
        print(f"[error] {data.get('error')}")


In [ ]:
resp_ft_stream = client.synthesize_finetuned_stream(
    text="""Najeriya ƙasa ce mai yalwa da al’adu, harsuna, da ƙabilu masu yawa, tana ɗauke da tarihin da ya haɗa mutane daban-daban cikin manufa guda. Daga arewa mai faɗaɗɗen hamada zuwa kudu mai arzikin gandun daji, ƙasar na cike da ni’imomi da baiwa ta halitta. Mutanen Najeriya suna da jajircewa, basira, da ƙwazo wajen fuskantar sauye-sauyen rayuwa. Duk da kalubale, ana ganin ƙarfin matasa, cigaban fasaha, da ƙoƙarin haɗin kai suna haskaka makoma mai kyau. Najeriya na ci gaba da ƙarfafa murya a nahiyar Afirka, tana nuni da dama da fata ga al’umma masu buri da mafarkai.""",
    model_name="chukypedro/hausa-tts-400m-0.3-pt",
    speaker_id="bashir",
    device="cpu",
    output_filename="tts_naijalingo_sample.wav",
)

resp_ft_stream

In [ ]:
# Download the finetuned output from the service and play locally
from IPython.display import Audio as APlay
import soundfile as sf

local_ft = download_tts_file(resp_ft["file_path"], base_url=BASE_URL)
audio, sr = sf.read(str(local_ft))
APlay(audio, rate=sr)


## Open-source MMS TTS: synthesize


In [ ]:
# Yoruba example
yoruba_resp_mms = client.synthesize_open_source(
    text="Ẹ káàrọ̀ o, báwo ni ìlera rẹ̀?",
    language="yoruba",   # 'hausa' | 'yoruba' | 'pidgin'
    device="cpu",
    output_filename="tts_mms_yoruba.wav",
)
print(yoruba_resp_mms)
print("--------------------------------")

# Hausa example
hausa_resp_mms = client.synthesize_open_source(
    text="Ina kwana, lafiya kalau?",
    language="hausa",   # 'hausa' | 'yoruba' | 'pidgin'
    device="cpu",
    output_filename="tts_mms_hausa.wav",
)
print(hausa_resp_mms)
print("--------------------------------")

# Pidgin English example
pidgin_resp_mms = client.synthesize_open_source(
    text="Good morning, how your body dey?",
    language="pidgin",   # 'hausa' | 'yoruba' | 'pidgin'
    device="cpu",
    output_filename="tts_mms_pidgin.wav",
)
print(pidgin_resp_mms)
print("--------------------------------")



In [ ]:
# Download the MMS output from the service and play locally
from IPython.display import Audio as APlay
import soundfile as sf

# Yoruba example
yoruba_local_mms = download_tts_file(yoruba_resp_mms["file_path"], base_url=BASE_URL)
audio, sr = sf.read(str(yoruba_local_mms))
APlay(audio, rate=sr)

# Hausa example
hausa_local_mms = download_tts_file(hausa_resp_mms["file_path"], base_url=BASE_URL)
audio, sr = sf.read(str(hausa_local_mms))
hausa_local_mms

# Pidgin English example
pidgin_local_mms = download_tts_file(pidgin_resp_mms["file_path"], base_url=BASE_URL)
audio, sr = sf.read(str(pidgin_local_mms))
pidgin_local_mms



## Gemini TTS: synthesize (requires GEMINI_API_KEY on service)


In [ ]:
hausa_text = """
        "Najeriya ƙasa ce mai yalwa da al’adu, harsuna, da ƙabilu masu yawa, "
        "tana ɗauke da tarihin da ya haɗa mutane daban-daban cikin manufa guda. "
        "Daga arewa mai faɗaɗɗen hamada zuwa kudu mai arzikin gandun daji, ƙasar "
        "na cike da ni’imomi da baiwa ta halitta. Mutanen Najeriya suna da jajircewa, "
        "basira, da ƙwazo wajen fuskantar sauye-sauyen rayuwa. Duk da kalubale, ana "
        "ganin ƙarfin matasa, cigaban fasaha, da ƙoƙarin haɗin kai suna haskaka makoma "
        "mai kyau. Najeriya na ci gaba da ƙarfafa murya a nahiyar Afirka, tana nuni "
        "da dama da fata ga al’umma masu buri da mafarkai."
       """
       
yoruba_text = """
    “Nàìjíríà jẹ́ orílẹ̀-èdè tí ó kún fún àṣà, èdè, àti ọ̀pọ̀lọpọ̀ ẹ̀yà, 
    tó sì ní ìtàn tí ó so àwọn ènìyàn oríṣìíríṣìí pọ̀ sínú èrò kan ṣoṣo. 
    Látọ̀dọ̀ àríwá tí ó ní aginjù púpò sí gúúsù tí ó kún fún igbo alárinrin, 
    orílẹ̀-èdè náà kún fún ìbùkún àti ẹ̀bùn àdàní. Àwọn ará Nàìjíríà ní 
    ìfaradà, ọgbọ́n, àti ìsapá nígbà gbogbo tí wọ́n bá dojú kọ ìyípadà ayé. 
    Bí ó tilẹ̀ jẹ́ pé ìṣòro wà, agbára àwọn ọ̀dọ́, ìlera imọ̀ ẹ̀rọ, àti 
    ìsapaya láti ṣiṣẹ́ pọ̀ ń tàn ìmólẹ̀ sí ọjọ́ ọ̀la rere. Nàìjíríà ń tẹ̀síwájú 
    ní agbára rẹ̀ ní Áfíríkà, tí ó fi ànfààní, ireti, àti ànfààní tuntun hàn 
    sí gbogbo ènìyàn tó ní ìfẹ́ sí ìlọsíwájú.”

    """

igbo_text = """

"""

In [ ]:
try:
    resp_g = client.synthesize_gemini(
        text=yoruba_text,
        language="yoruba",      # 'yoruba' | 'igbo' | 'hausa' | 'pidgin'
        voice_name="Zephyr",
        style_prompt=None,
        output_filename="tts_gemini_pidgin.wav",
    )
    resp_g
except Exception as e:
    print("Gemini call failed:", e)
    print("Ensure GEMINI_API_KEY is set on the speech service.")


In [ ]:
# Download the finetuned output from the service and play locally
from IPython.display import Audio as APlay
import soundfile as sf

local_g = download_tts_file(resp_g["file_path"], base_url=BASE_URL)
audio, sr = sf.read(str(local_g))
APlay(audio, rate=sr)
